In [1]:
import cv2
import numpy as np
import os
import math
from matplotlib import pyplot as plt
from IPython.display import clear_output
%matplotlib inline

In [2]:
def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        w_rm = int(0.2 * w / 2)
        faces.append(image[y: y + h, x + w_rm: x + w - w_rm])
         
    return faces

def detect_face(frame):
    detector = cv2.CascadeClassifier("xml/frontal_face.xml")

    scale_factor = 1.2
    min_neighbors =5
    min_size = (30,30)
    flags = cv2.CASCADE_SCALE_IMAGE

    faces = detector.detectMultiScale(frame,scaleFactor=scale_factor,
                                     minNeighbors=min_neighbors,
                                     minSize=min_size,
                                     flags=flags)
    
    return faces

def gray_scale(images):
    gray_images = []
    for image in images:
        is_color = len(image.shape) == 3 
        if is_color:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_images.append(image)
    return gray_images

def resize(images,size=(90,90)):
    image_resize = []
    
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,
                                     interpolation=cv2.INTER_AREA)
        else:
            img_size = cv2.resize(image,size,
                                     interpolation=cv2.INTER_CUBIC)
        image_resize.append(img_size)
        
    return image_resize

def normalize_faces(frame, faces_coord):
    faces = cut_faces(frame, faces_coord)
    #faces = normalize_intensity(faces)
    faces = gray_scale(faces)
    faces = resize(faces)
    return faces

def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        w_rm = int(0.2 * w / 2) 
        cv2.rectangle(image, (x + w_rm, y), (x + w - w_rm, y + h), 
                              (0,0,255),2)

In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

# Open a new thread to manage the external cv2 interaction
cv2.startWindowThread()

def plt_show(image,title=""):
    if len(image.shape)==3:
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.axis('off')
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

class FaceDetector():
    def __init__(self,xml_path):
        self.classifier = cv2.CascadeClassifier(xml_path)

    def detect(self,image):
        scale_factor = 1.2
        min_neighbors=5
        min_size=(30,30)
        flags = cv2.CASCADE_SCALE_IMAGE

        faces_coord = self.classifier.detectMultiScale(image,
                                                       scaleFactor=scale_factor,
                                                       minNeighbors=min_neighbors,
                                                       minSize = min_size,
                                                       flags=flags)

        return faces_coord


class VideoCamera():
    def __init__(self,index=1):
        self.video = cv2.VideoCapture(index)
        self.index=index
        print self.video.isOpened()

    def __del__(self):
        self.video.release()

    def get_frame(self,grayscale=False):
        ret,frame = self.video.read()

        if grayscale:
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        return frame

SyntaxError: invalid syntax (<ipython-input-3-1fafa0542df8>, line 43)

In [ ]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
    #people = [person for person in os.listdir("Male_female/")]
    people = [person for person in os.listdir("trainingData/")]
    #people = [person for person in os.listdir("people/")]
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir("trainingData/" + person):
            if image.endswith('.jpg'):
                images.append(cv2.imread("trainingData/" + person + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

In [ ]:
images, labels, labels_dic = collect_dataset()

In [ ]:
print len(images)
print type(images)

In [ ]:
X_train=np.asarray(images)

In [ ]:
X_train.shape

In [ ]:
train=X_train.reshape(len(X_train),-1)

In [ ]:
train.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca1 = PCA()
pca1.fit(train)
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
pca1 = PCA(n_components=400)
new_train=pca1.fit_transform(train)

In [ ]:
new_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold,cross_val_score

models = [KNeighborsClassifier(),LogisticRegression(),SVC(kernel='linear',C=.001)]

kf = KFold(n_splits=10,shuffle=True)


for model in models:
    result = cross_val_score(model,new_train,labels,cv=5,scoring='accuracy')
    
    print np.mean(result)
    
    

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:
param_grid = {'C':[.0001,.001,.01,1,10],
              'kernel':['linear','rbf'],
              'gamma':[.0001,.001,.01,.1,10]
             }

In [ ]:
kf=KFold(n_splits=10,shuffle=True)

In [ ]:
gs_svc = GridSearchCV(SVC(),param_grid=param_grid,cv=kf,scoring='accuracy')

In [ ]:
gs_svc.fit(new_train,labels)

In [ ]:
gs_svc.best_score_

In [ ]:
gs_svc.best_params_

In [ ]:
clf_svc = gs_svc.best_estimator_

In [ ]:
clf_svc

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(new_train,labels,test_size=.20,random_state=0,
                                              shuffle=True)

In [ ]:
clf_svc.fit(X_train,y_train)

In [ ]:


clf_svc.score(X_train,y_train)

In [ ]:
clf_svc.score(X_test,y_test)

### Adaboost

In [ ]:
param = {'algorithm':['SAMME.R','SAMME'],
        'base_estimator':[RandomForestClassifier(),None],}

In [ ]:
gs_ada = GridSearchCV(AdaBoostClassifier(random_state=1),param_grid=param,cv=kf,scoring='accuracy')

In [ ]:
gs_ada.fit(new_train,labels)

In [ ]:
gs_ada.best_score_

In [ ]:
gs_ada.best_params_

In [4]:
clf_ada = gs_ada.best_estimator_

NameError: name 'gs_ada' is not defined

In [ ]:
clf_ada.fit(X_train,y_train)

In [5]:
clf_ada.score(X_test,y_test)

NameError: name 'clf_ada' is not defined

In [6]:
clf_ada.score(X_train,y_train)

NameError: name 'clf_ada' is not defined

### save & load model

In [7]:
import pickle

filename = 'svc_model.pkl'
f=open(filename, 'wb')
pickle.dump(clf_svc,f)
 

f.close()


NameError: name 'clf_svc' is not defined

In [8]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

EOFError: Ran out of input

### Live face classification

In [9]:
svc1=gs_svc.best_estimator_
svc1.probability=True
svc1.fit(new_train,labels)

NameError: name 'gs_svc' is not defined

In [10]:
detector = FaceDetector("xml/frontal_face.xml")
webcam = VideoCamera(0)

NameError: name 'FaceDetector' is not defined

In [11]:
cv2.startWindowThread()
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    frame = webcam.get_frame()
    
    
    faces_coord = detector.detect(frame) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord) 
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            test = pca1.transform(face.reshape(1,-1))    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print confidence
            print prob
           
            
            
            pred = svc1.predict(test)
            print pred,pred[0]
           
            name=labels_dic[pred[0]].capitalize()
            print name
            
            #pred = labels_dic[pred[0]].capitalize()
            #threshold = .50
            if confidence > 1:
             
                
                cv2.putText(frame, name,
                            (faces_coord[i][0]
                             , faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
            else:
                
                
                cv2.putText(frame, name,
                            (faces_coord[i][0], faces_coord[i][1]),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
           
                
                
            #cv2.putText(frame,name,(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
            
               
           #cv2.putText(frame,'Unknown',(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),
                cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,
                cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) & 0xFF == 27:
        
        cv2.destroyAllWindows()
        break

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(confidence)? (<ipython-input-11-69a355c7d2ff>, line 24)

In [12]:
del webcam

NameError: name 'webcam' is not defined